In [1]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

from pydantic import BaseModel
import json

# API Key 설정
os.environ["OPENAI_API_KEY"] = "SECRET KEY"

model = ChatOpenAI(temperature=0)

In [2]:
news = pd.read_csv('news_with_labels.csv', usecols = ['Text', 'Completion'])

class StockImpact(BaseModel):
    sector: str
    impact: str
    reason: str

# 모델 정의        
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# system prompt
system_message_prompt = """
너는 뉴스 기사를 읽고 해당 기사가 영향을 미칠 주식 섹터를 탐지하는 ai 어시스턴트야.
"""
system_message_prompt_template = SystemMessagePromptTemplate.from_template(system_message_prompt)

# human prompt
human_message_prompt = """
{content}는 조회수와 댓글이 많은 뉴스로, 사람들의 관심이 큰 이슈를 다룬 뉴스야.
다음 <기준>에 따라 이 뉴스가 영향을 미칠 주식 테마와 그 이유를 <형식>에 따라 제시해줘.
테마는 테마 목록인 <테마 목록> 안에 있는 테마만 제시해줘.

<기준>
먼저 뉴스 기사를 읽고 영향을 미칠 만한 주식 섹터를 선정하고 영향과 이유를 알려줘..
해당 기사가 직접적으로 영향을 미치는 섹터 뿐만 아니라, 간접적으로 영향을 미칠 만한 섹터도 추천해줘.
단, 이 경우 명확하게 인과관계가 존재하는 섹터를 추천해줘야해.
예를 들면, 코로나19 전염병이 유행할 때 사람들이 집에 있는 시간이 늘어나면서 ott 플렛폼의 수익이 크게 늘어났고, 관련 섹터의 주가가 상승했어.
확실하게 영향을 미칠 것으로 예상되는 섹터만 제시해줘.
해당 섹터에 영향을 미치게 되는 이유를 상세히 설명해줘.
영향을 미칠 만한 섹터가 여러 개인 경우 모두 제시해줘.

<형식>
sector : [주식 섹터]
impact : [긍정] 또는 [부정]
reason : [이유]

<테마 목록>
'마리화나(대마)', '코로나19(카모스타트)', '의료AI', '전력설비', '선박평형수 처리장치', '전선', '조선기자재', '비만치료제', '구충제(펜벤다졸, 이버멕틴 등)', '조선', '탈모 치료', '초전도체', '미용기기', '피팅(관이음쇠)/밸브', '케이블TV SO/MSO', '원격진료/비대면진료(U-Healthcare)', '치매', '보톡스(보툴리눔톡신)', '냉각시스템(액침냉각 등)', '면역항암제', '웹툰', 'LNG(액화천연가스)', '전력저장장치(ESS)', '밥솥', '로봇(산업용/협동로봇 등)', '스마트팩토리(스마트공장)', '의료기기', 'mRNA(메신저 리보핵산)', '자동차 대표주', '제약업체', '코로나19(치료제/백신 개발 등)', '화장품', 'RFID(NFC 등)', '2019 하반기 신규상장', '수산', '전기차(충전소/충전기)', '2019 상반기 신규상장', '코로나19(스푸트니크V)', '바이오시밀러(복제 바이오의약품)', '코로나19(덱사메타손)', '우주항공산업(누리호/인공위성 등)', '마이코플라스마 폐렴', 'CCTV＆DVR', '온실가스(탄소배출권)', '드론(Drone)', '코로나19(나파모스타트)', '사료', '방위산업/전쟁 및 테러', '스마트그리드(지능형전력망)', '구제역/광우병 수혜', '타이어', '공기청정기', '면세점', '무선충전기술', '2024 상반기 신규상장', 'LED', '태양광에너지', '음식료업종', '주류업(주정, 에탄올 등)', '카지노', '남-북-러 가스관사업', '코로나19(진단/치료제/백신 개발 등)', '터치패널(스마트폰/태블릿PC 등)', '해저터널(지하화/지하도로 등)', '국내 상장 중국기업', '제습기', '유전자 치료제/분석', 'LCD 부품/소재', 'IT 대표주', '2023 하반기 신규상장', '백신/진단시약/방역(신종플루, AI 등)', '철강 중소형', '온디바이스 AI', 'UAM(도심항공모빌리티)', '겨울', '보안주(물리)', '그래핀', '마이크로바이옴', '조림사업', '환율하락 수혜', '고령화 사회(노인복지)', '아스콘(아스팔트 콘크리트)', '제대혈', '리츠(REITs)', '코로나19(음압병실/음압구급차)', '엔터테인먼트', '페인트', '손해보험', '마스크', '소모성자재구매대행(MRO)', '지능형로봇/인공지능(AI)', '화학섬유', '3D 프린터', '전기차', '제지', '코로나19(진단키트)', '2021 하반기 신규상장', '수소차(연료전지/부품/충전소 등)', '황사/미세먼지', '모바일콘텐츠(스마트폰/태블릿PC)', '철도', '차량용블랙박스', '기업인수목적회사(SPAC)', '마이크로 LED', '통신', '증권', '건설기계', '전기자전거', 'MVNO(가상이동통신망사업자)', '지주사', '2021 상반기 신규상장', '강관업체(Steel pipe)', '항공기부품', '2020 상반기 신규상장', '도시가스', '패션/의류', '콜드체인(저온 유통)', '골판지 제조', '모듈러주택', '자동차부품', '자동차부품', '비료', '건강기능식품', '2020 하반기 신규상장', '원자력발전', '핵융합에너지', '가상현실(VR)', '자율주행차', 'HBM(고대역폭메모리)', '리모델링/인테리어', '스마트카(SMART CAR)', '고체산화물 연료전지(SOFC)', '시멘트/레미콘', '종합상사', 'K-뉴딜지수(바이오)', '日제품 불매운동(수혜)', '반도체 재료/부품', '코로나19(렘데시비르)', '뉴로모픽 반도체', '4차산업 수혜주', '치아 치료(임플란트 등)', '재택근무/스마트워크', '키오스크(KIOSK)', '은행', '반도체 대표주(생산)', '자원개발', '여름', '홈쇼핑', '모더나(MODERNA)', '비철금속', '음원/음반', '전자결제(전자화폐)', '종합 물류', 'STO(토큰증권 발행)', '자전거', '일자리(취업)', '골프', '항공/저가 항공사(LCC)', '토스(toss)', '메르스 코로나 바이러스', '메르스 코로나 바이러스', '풍력에너지', '쿠팡(coupang)', 'SSD', '스포츠행사 수혜(올림픽, 월드컵 등)', '네옴시티', '2022 상반기 신규상장', '셰일가스(Shale Gas)', 'GTX(수도권 광역급행철도)', '줄기세포', 'DMZ 평화공원', '빈대', '생명보험', '아프리카 돼지열병(ASF)', '테마파크', '건설 중소형', '5G(5세대 이동통신)', '모바일게임(스마트폰)', '농업', '2023 상반기 신규상장', '엔젤산업', '시스템반도체', '3D 낸드(NAND)', '통신장비', '증강현실(AR)', '교육/온라인 교육', '아이폰', '슈퍼박테리아', '영화', 'OLED(유기 발광 다이오드)', '렌터카', '스마트홈(홈네트워크)', '삼성페이', '스마트폰', '니켈', '태블릿PC', '카메라모듈/부품', '백화점', '플렉서블 디스플레이', '출산장려정책', 'CXL(컴퓨트익스프레스링크)', '출산장려정책', '엠폭스(원숭이두창)', '원자력발전소 해체', '모바일솔루션(스마트폰)', 'LPG(액화석유가스)', '영상콘텐츠', '맥신(MXene)', '여행', 'NI(네트워크통합)', '전자파', 'MLCC(적층세라믹콘덴서)', '메타버스(Metaverse)', '2022 하반기 신규상장', '반도체 장비', '화이자(PFIZER)', '2차전지(장비)', '재난/안전(지진 등)', 'PCB(FPCB 등)', '사물인터넷', '태풍 및 장마', '철강 주요종목', '두나무(Dunamu)', '윤활유', 'SI(시스템통합)', '갤럭시 부품주', '휴대폰부품', '클라우드 컴퓨팅', 'LCD장비', '日 수출 규제(국산화 등)', '음성인식', '소매유통', '게임', '마이데이터', 'LED장비', '해운', '마켓컬리(kurly)', '리비안(RIVIAN)', '인터넷은행', '2차전지(소재/부품)', '2차전지(소재/부품)', '카카오뱅크(kakao BANK)', '폐기물처리', '페라이트', '편의점', '공작기계', '양자암호', '2차전지(LFP/리튬인산철)', '2차전지', '창투사', '석유화학', '광고', '애플페이', '유리 기판', '2차전지(생산)', 'K-뉴딜지수(인터넷)', '4대강 복원', '화폐/금융자동화기기(디지털화폐 등)', 'LCD BLU제조', '코로나19(혈장치료/혈장치료제)', '크래프톤 관련주', '북한 광물자원개발', '건설 대표주', '폴더블폰', '2차전지(전고체)', '정유', '핀테크(FinTech)', '폐배터리', '바이오인식(생체인식)', '2024 하반기 신규상장', 'NFT(대체불가토큰)', '탄소나노튜브(CNT)', 'SNS(소셜네트워크서비스)', '육계', 'AI 챗봇(챗GPT 등)', '보안주(정보)', '백신여권', '제4이동통신', '남북경협', 'K-뉴딜지수(게임)', '요소수', '캐릭터상품', '블록체인', '희귀금속(희토류 등)', '가상화폐(비트코인 등)', 'K-뉴딜지수(2차전지)', '리튬', '인터넷 대표주', '야놀자(Yanolja)'
"""

human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_message_prompt)

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt_template, human_message_prompt_template])

# JSON Output Parser 설정
json_output_parser = JsonOutputParser()

def get_stock_impact(content):
    formatted_prompt = chat_prompt_template.format_messages(content=content)
    response = model.invoke(formatted_prompt)
    return response.content

# 뉴스 기사 내용을 입력으로 받아 주식 영향을 분석
for i in range(1):
    content = news.iloc[i, 0]
    impact_response = get_stock_impact(content)
    news.at[i, 'Completion'] = impact_response

In [3]:
news.to_csv('news_with_label.csv')